# Section 1: Raw Sales Sources 

## Context

An organization receives **sales transactions** from **two independent raw sources**.  
Both sources provide the same set of fields, but they are ingested into separate raw tables so that:
- the original incoming data is preserved as-is, and
- each source can be audited independently using load metadata.

The raw data is stored in a MySQL database named **`source_db`**.


## Raw Source Tables

The database contains two raw tables:

1. **`sales_raw_source1`**  
2. **`sales_raw_source2`**

Both tables have an identical structure and represent **sales transaction records**.


## Data Dictionary

Each raw sales record includes:

- **order_id** *(VARCHAR(20))*  
  Identifier of the order/transaction (not guaranteed to be unique unless enforced separately).

- **customer_name** *(VARCHAR(100))*  
  Name of the customer who placed the order.

- **city** *(VARCHAR(50))*  
  Customer’s city.

- **email** *(VARCHAR(100))*  
  Customer’s email address.

- **product_name** *(VARCHAR(100))*  
  Name of the product sold.

- **product_category** *(VARCHAR(50))*  
  Category of the product.

- **sale_date** *(DATE)*  
  Date on which the sale occurred.

- **quantity** *(INT)*  
  Number of units purchased in the record.

- **unit_price** *(DECIMAL(10,2))*  
  Price per unit.

- **load_timestamp** *(TIMESTAMP)*  
  Timestamp indicating when this record was loaded/ingested into the raw table (useful for auditing and incremental loads).


### Problem to Solve

You are given two raw sales sources stored as MySQL tables. Your job is to treat these as **incoming raw feeds** and prepare them for downstream analytics.

The requirements are:

1. **Store** each source independently in raw tables (`sales_raw_source1`, `sales_raw_source2`).
2. **Preserve** the original incoming fields without modification.
3. **Track** ingestion time using `load_timestamp` for auditability.


## Source Verify

Validate row counts and preview a few rows from each source table.

In [ ]:
SELECT 'source1' AS src, COUNT(*) AS rows_count FROM source_db.sales_raw_source1
UNION ALL
SELECT 'source2' AS src, COUNT(*) AS rows_count FROM source_db.sales_raw_source2;

In [ ]:
SELECT * FROM source_db.sales_raw_source1 ORDER BY load_timestamp, order_id;

In [ ]:
SELECT * FROM source_db.sales_raw_source2 ORDER BY load_timestamp, order_id;

# Section 2 - Build the Bronze Layer

## Context

The goal of the **Bronze layer** is to act as the **raw landing layer inside the DWH**, where incoming sales data is captured in its original form, with minimal transformation.

In this simplified training pipeline:
- Sales data arrives from **two operational source tables** in `source_db`
- Both sources have the same structure but represent different **source systems**
- The Bronze layer **merges** both sources into **one consolidated Bronze table**
- To keep the flow easy to teach, Bronze uses a **full load** pattern:
  - `TRUNCATE` the Bronze table
  - `INSERT` fresh data from both sources

## Raw Source Tables

Bronze reads raw sales records from these two source tables:

1. `source_db.sales_raw_source1`  
2. `source_db.sales_raw_source2`

In Bronze, these sources are tagged using a `source_system` column:
- `flipkart` → records from `sales_raw_source1`
- `amazon` → records from `sales_raw_source2`

## Data Dictionary

Bronze stores consolidated raw records in: `bronze_db.bronze_sales_raw`

| Column | Description |
|---|---|
| `source_system` | Identifies the source system of the record (`flipkart` or `amazon`). |
| `order_id` | Business order identifier for the transaction. |
| `customer_name` | Customer name captured in the raw feed. |
| `city` | Customer city from the raw feed (may be NULL or inconsistent). |
| `email` | Customer email as received (may contain invalid formats). |
| `product_name` | Product name sold in the transaction. |
| `product_category` | Product category from the raw feed. |
| `sale_date` | Date of the sale transaction. |
| `quantity` | Units purchased in the transaction. |
| `unit_price` | Price per unit for the item. |
| `load_timestamp` | Timestamp when the record was loaded into the source table (audit field). |

## Problems to Solve in Bronze

Implement the Bronze ingestion layer with the following requirements:

1. **Create the Bronze database and table**
   - Database: `bronze_db`
   - Table: `bronze_sales_raw`

2. **Consolidate both source tables into one Bronze table**
   - Load all records from `source_db.sales_raw_source1` and tag them as `source_system = 'flipkart'`
   - Load all records from `source_db.sales_raw_source2` and tag them as `source_system = 'amazon'`

3. **Use a full refresh strategy (full load)**
   - `TRUNCATE` the Bronze table before every load run
   - Re-load data using `INSERT INTO ... SELECT ...` from both sources

4. **Preserve raw data as-is**
   - Do not cleanse, standardize, or deduplicate in Bronze
   - The only enrichment allowed is adding `source_system` for traceability

5. **Support traceability and auditability**
   - Keep `load_timestamp` intact to support later analysis on ingestion timing and data freshness


## Bronze Verify

Check how many rows came from each source and preview the merged data.

In [ ]:
SELECT source_system, COUNT(*) AS rows_count
FROM bronze_db.bronze_sales_raw
GROUP BY source_system;

In [ ]:
SELECT * FROM bronze_db.bronze_sales_raw ORDER BY load_timestamp, source_system, order_id;

# Section 3 - Build the Silver Layer

## Context

The purpose of the **Silver layer** is to store **cleansed and standardized** sales data.

Silver reads consolidated raw records from the Bronze table (`bronze_db.bronze_sales_raw`) and applies business cleansing rules before writing to a curated Silver table.

In this simplified training pipeline, Silver performs three key actions:
1. **Standardizes city values** (business standardization)
2. **Filters out records that should not be promoted** (business rule example: exclude Business plans to stop selling `Beauty` products, and they need not be appearing in the analytics)
3. **Deduplicates records by `order_id`** (keeps the latest row based on `load_timestamp`)

## Input Table

Silver reads from:

- `bronze_db.bronze_sales_raw`

This table contains raw consolidated records from multiple source systems, along with `load_timestamp` for auditability.

## Output Table

Silver writes cleansed records into:

- `silver_db.silver_sales_clean`

This table enforces **one row per `order_id`** using:
- `order_id` as the **PRIMARY KEY**
- deduplication logic during load to ensure uniqueness

## Data Dictionary (Silver Output Table)

| Column | Description |
|---|---|
| `order_id` | Unique order identifier in Silver (PRIMARY KEY). |
| `source_system` | Source system from Bronze (e.g., `flipkart`, `amazon`). |
| `customer_name` | Customer name as received from Bronze. |
| `city` | Standardized city value after Silver cleansing rules. |
| `email` | Email after trimming spaces (basic standardization). |
| `product_name` | Product name sold. |
| `product_category` | Product category (excluding `Beauty` in Silver). |
| `sale_date` | Date of sale transaction. |
| `quantity` | Units purchased. |
| `unit_price` | Price per unit. |
| `load_timestamp` | Timestamp representing the latest ingested version of the record kept in Silver. |

## Problems to Solve in Silver

Implement the Silver layer with the following requirements:

1. **Create the Silver database and curated table**
   - Database: `silver_db`
   - Table: `silver_sales_clean`

2. **Standardize city values**
   Apply the following business rules for `city`:
   - If `city` is **NULL**, **blank**, or equals the string **`null`** (case-insensitive), default it to **`Delhi`**
   - If `city` is `NY`, standardize it to **`New York`**
   - If `city` is `Hyd` or `HYD` (case variations), standardize it to **`Hyderabad`**
   - Otherwise, store the trimmed city value

3. **Standardize email values**
   - Apply `TRIM(email)` to remove leading/trailing spaces

4. **Filter out records not eligible for promotion**
   - Remove rows where `product_category = 'Beauty'`
   - Keep rows where `product_category` is NULL or not equal to `Beauty`

5. **Deduplicate by `order_id`**
   - If multiple rows share the same `order_id`, keep **only the latest record** based on `load_timestamp`
   - Use a window function such as:
     - `ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY load_timestamp DESC)`

6. **Use a full refresh strategy (full load)**
   - `TRUNCATE` the Silver table before each load run
   - Reload the cleansed, filtered, deduplicated dataset from Bronze


## Silver Verify

Validate row counts and inspect the cleaned output.

In [ ]:
SELECT COUNT(*) AS silver_rows FROM silver_db.silver_sales_clean;

In [ ]:
SELECT COUNT(*) AS missing_city_after_cleaning
FROM silver_db.silver_sales_clean
WHERE city IS NULL OR TRIM(city) = '';

In [ ]:
SELECT COUNT(*) AS beauty_rows_in_silver
FROM silver_db.silver_sales_clean
WHERE product_category = 'Beauty';

In [ ]:
SELECT * FROM silver_db.silver_sales_clean ORDER BY load_timestamp, order_id;

# Section 4 - Build the Gold Layer

## Context

The purpose of the **Gold layer** is to store **business-ready, analytics-friendly** data that supports reporting and BI use cases.

In this pipeline, the Gold layer models the cleansed Silver data into a **simple star schema** with:
- `dim_customer`
- `dim_product`
- `fact_sales`

- Customer is uniquely identified by **email**
- If the same email appears with different attributes over time, keep the **latest** version using `load_timestamp`
- The fact table stores a derived measure:  
  `total_amount = quantity * unit_price`

## Input Table

Gold reads from:

- `silver_db.silver_sales_clean`

This table already contains cleansed and deduplicated sales records (one row per `order_id`).

## Star Schema Tables (Gold Output)

Gold writes into these three tables in `gold_db`:

1. **Customer Dimension**: `gold_db.dim_customer`  
2. **Product Dimension**: `gold_db.dim_product`  
3. **Sales Fact**: `gold_db.fact_sales`

## Data Dictionary

### `dim_customer`

| Column | Description |
|---|---|
| `customer_key` | Surrogate key (AUTO_INCREMENT PRIMARY KEY). |
| `email` | Natural key for customer (UNIQUE). |
| `customer_name` | Latest known customer name for the email. |
| `city` | Latest known city for the email. |
| `last_seen_ts` | Latest `load_timestamp` seen for the email (used to keep the newest record). |

### `dim_product`

| Column | Description |
|---|---|
| `product_key` | Surrogate key (AUTO_INCREMENT PRIMARY KEY). |
| `product_name` | Product name. |
| `product_category` | Product category. |
| `(product_name, product_category)` | Composite natural key (UNIQUE). |

### `fact_sales`

| Column | Description |
|---|---|
| `sales_key` | Surrogate key (AUTO_INCREMENT PRIMARY KEY). |
| `order_id` | Business order identifier (UNIQUE). |
| `customer_key` | Foreign key to `dim_customer.customer_key`. |
| `product_key` | Foreign key to `dim_product.product_key`. |
| `sale_date` | Date of sale. |
| `quantity` | Units purchased. |
| `unit_price` | Price per unit. |
| `total_amount` | Derived measure: `ROUND(quantity * unit_price, 2)`. |
| `load_timestamp` | Record timestamp carried from Silver (audit field). |

## Problems to Solve in Gold

Implement the Gold layer with the following requirements:

1. **Create the Gold database and star schema tables**
   - Database: `gold_db`
   - Tables: `dim_customer`, `dim_product`, `fact_sales`

2. **Load Customer Dimension (`dim_customer`)**
   - Customer is uniquely identified by `email`
   - Only load records where `email` is not NULL and not blank after trimming
   - If multiple records exist for the same email, keep the **latest** record using:
     - `ROW_NUMBER() OVER (PARTITION BY email ORDER BY load_timestamp DESC)`
   - Store the latest timestamp in `last_seen_ts`

3. **Load Product Dimension (`dim_product`)**
   - Product uniqueness is defined by `(product_name, product_category)`
   - Insert distinct combinations from Silver
   - Only load records where both `product_name` and `product_category` are not NULL

4. **Load Sales Fact (`fact_sales`)**
   - One fact row per `order_id` (order_id is UNIQUE)
   - For each Silver record:
     - Look up the matching customer using `email` → `dim_customer.customer_key`
     - Look up the matching product using `(product_name, product_category)` → `dim_product.product_key`
   - Populate measures and audit fields:
     - `total_amount = ROUND(quantity * unit_price, 2)`
     - carry forward `load_timestamp`

5. **Use a full refresh strategy (full load)**
   - `TRUNCATE` `fact_sales`, `dim_customer`, and `dim_product`
   - Rebuild dimensions first, then load the fact table


## Gold Verify

In [ ]:
SELECT COUNT(*) AS customers FROM gold_db.dim_customer;
SELECT COUNT(*) AS products  FROM gold_db.dim_product;
SELECT COUNT(*) AS facts     FROM gold_db.fact_sales;

In [ ]:
SELECT * FROM gold_db.fact_sales ORDER BY sale_date, order_id;

# Section 5: Analytics on the Gold Layer

## Write queries to answer the following business questions

### Revenue by City

### Revenue by Category

### Daily Revenue Trend

# Section 6: ETL Testing (Quality Assurance) 
- Testing reliability of the ETL pipeline

## Context

You have a **MySQL-based DWH pipeline** with three layers:

- **Bronze**: consolidated raw landing table
- **Silver**: cleansed + standardized table (business rules applied)
- **Gold**: star schema (`dim_customer`, `dim_product`, `fact_sales`) for analytics

To ensure the pipeline is reliable, you are implementing a **QA (Quality Assurance) test framework** that runs validations across Bronze, Silver, and Gold, and stores PASS/FAIL outcomes for each pipeline run in a centralized QA table.

Each execution of the QA suite generates a unique `test_run_id` (timestamp-based) so that all test results from that run can be grouped and audited.
- To get test_run_id, use the statement `SET @test_run_id = DATE_FORMAT(NOW(), '%Y%m%d_%H%i%s');`

## Input Table

QA checks read from the following DWH tables:

- `bronze_db.bronze_sales_raw`
- `silver_db.silver_sales_clean`
- `gold_db.fact_sales`

QA results are written into:

- `qa_db.test_results`

## Data Dictionary

### `qa_db.test_results`

| Column | Description |
|---|---|
| `test_run_id` | Unique identifier for a single run of the QA suite (e.g., `YYYYMMDD_HHMMSS`). |
| `test_name` | Name of the validation being executed. |
| `status` | PASS/FAIL outcome of the test. |
| `actual_value` | Numeric value produced by the test (typically a count of rule violations). |
| `expected_desc` | Human-readable description of what is expected (e.g., “0 expected”). |
| `details` | Short explanation of what the test validates. |
| `run_ts` | Timestamp when the test result row was inserted (default: current timestamp). |

## Problems to Solve

Implement a QA suite that inserts one row per test into `qa_db.test_results` for the current `test_run_id`.

### 1) Bronze: Duplicate `order_id` Check
- **Goal:** Detect `order_id` values that appear more than once in Bronze.
- **Expected:** `0` duplicates.
- **PASS/FAIL:** PASS if count = 0, else FAIL.

### 2) Silver: Null/Blank City Check
- **Goal:** Ensure `city` is not NULL or blank after Silver standardization.
- **Expected:** `0` rows with `city IS NULL OR TRIM(city) = ''`.
- **PASS/FAIL:** PASS if count = 0, else FAIL.

Example Query:
```sql
INSERT INTO qa_db.test_results (test_run_id, test_name, status, actual_value, expected_desc, details)
SELECT
  @test_run_id,
  'Silver: Beauty exclusion rule',
  CASE WHEN COUNT(*) = 0 THEN 'PASS' ELSE 'FAIL' END,
  COUNT(*),
  '0 expected',
  'Beauty rows should not be promoted to Silver'
FROM silver_db.silver_sales_clean
WHERE product_category = 'Beauty';
```

### 3) Silver: Beauty Products Exclusion Rule
- **Goal:** Confirm that `product_category = 'Beauty'` does not exist in Silver.
- **Expected:** `0` rows.
- **PASS/FAIL:** PASS if count = 0, else FAIL.

### 4) Silver: Invalid Email Format Check
- **Goal:** Identify invalid email values (example: `invalid-email`).
- **Rule (simplified):** Email must contain both `@` and `.`.
- **Expected:** `0` invalid emails.
- **PASS/FAIL:** PASS if count = 0, else FAIL.

### 5) Gold Fact: `total_amount` Calculation Check
- **Goal:** Validate `total_amount` matches `quantity * unit_price` within tolerance.
- **Rule:** `ABS(total_amount - (quantity * unit_price)) <= 0.01`.
- **Expected:** `0` mismatches.
- **PASS/FAIL:** PASS if count = 0, else FAIL.

### 6) Completeness: Silver → Gold Missing `order_id` Check
- **Goal:** Ensure every Silver `order_id` exists in Gold `fact_sales`.
- **Rule:** Silver left join fact; count rows where `fact.order_id IS NULL`.
- **Expected:** `0` missing order_ids.
- **PASS/FAIL:** PASS if count = 0, else FAIL.
